<a href="https://colab.research.google.com/github/Vandanasprabhu/Animal-Species-Recognition/blob/main/Animal_Species_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dir="/content/drive/MyDrive/Animals_Data/Test"
validation_dir="/content/drive/MyDrive/Animals_Data/Test"

In [ ]:
#import os
#os.listdir('/content/drive/MyDrive/Animals_Data/Test')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Animals_Data/Test'

In [ ]:
num_classes=26
image_size=(299,299)
batch_size=32
learning_rate=0.001
epochs=5

In [ ]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
valid_datagen=ImageDataGenerator(
    rescale=1./255)

In [ ]:
base_model=InceptionV3(weights='imagenet',include_top=False,input_shape=(image_size[0],image_size[1],3))

In [ ]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
predictions=Dense(num_classes,activation='softmax')(x)

In [ ]:
model=Model(inputs=base_model.input,outputs=predictions)

In [ ]:
for layer in base_model.layers:
  layer.trainable=False

In [ ]:
model.compile(optimizer=Adam(learning_rate=learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator=valid_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)



Found 1303 images belonging to 26 classes.
Found 1303 images belonging to 26 classes.


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=len(valid_generator)
)


Epoch 1/5
41/41 [==============================] - 748s 18s/step - loss: 0.6558 - accuracy: 0.8526 - val_loss: 0.0738 - val_accuracy: 0.9839
Epoch 2/5
41/41 [==============================] - 697s 17s/step - loss: 0.1128 - accuracy: 0.9693 - val_loss: 0.0401 - val_accuracy: 0.9885
Epoch 3/5
41/41 [==============================] - 685s 17s/step - loss: 0.0972 - accuracy: 0.9724 - val_loss: 0.0330 - val_accuracy: 0.9908
Epoch 4/5
41/41 [==============================] - 685s 17s/step - loss: 0.0390 - accuracy: 0.9870 - val_loss: 0.0229 - val_accuracy: 0.9908
Epoch 5/5
41/41 [==============================] - 688s 17s/step - loss: 0.0476 - accuracy: 0.9885 - val_loss: 0.0187 - val_accuracy: 0.9954


In [ ]:
model.save("/content/drive/MyDrive/inceptionv3_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model=tf.keras.models.load_model('/content/drive/MyDrive/inceptionv3_model.h5')

In [ ]:
class_labels=['Bear', 'Cat', 'Chicken', 'Dog', 'Duck', 'Elephant', 'Fish', 'Fox', 'Frog', 'Giraffe', 'Hippopotamus', 'Horse', 'Kangaroo', 'Koala', 'Lion', 'Monkey', 'Panda', 'Peacock', 'Penguin']
len(class_labels)

19

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import files
import ipywidgets as widgets
from IPython.display import display

In [ ]:
def preprocess_image(image_path):
  img=PIL.Image.open(image_path)
  img=img.resize((299,299))
  img=np.array(img)/255.0
  img=np.expand_dims(img,axis=0)
  return img



In [ ]:
def predict_animal(image_path):
  processed_image=preprocess_image(image_path)
  predictions=model.predict(processed_image)
  predicted_label=class_labels[np.argmax(predictions)]
  confidence=np.max(predictions)
  return predicted_label,confidence





from google.colab import files
uploaded=files.upload()

uploaded_file_path=list(uploaded.keys())[0]

prediction,confidence=predict_animal(uploaded_file_path)
print(confidence)

img=PIL.Image.open(uploaded_file_path)
plt.imshow(img)
plt.axis('off')
if (confidence<0.80):

    print("Data could not be found.Sorry for the inconvienence")

else:
    plt.title(f'Prediction:{prediction}\nConfidence:{confidence:.2%}')
    plt.show()
